[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RichmondAlake/agent_memory_course/blob/main/langmem/memory_augmented_agent_with_mongodb.ipynb)


In [12]:
%pip install -qU langmem langgraph langchain_voyageai langgraph-checkpoint-mongodb langgraph-store-mongodb pymongo requests pypdf langchain-community

Note: you may need to restart the kernel to use updated packages.


In [13]:
import getpass
import os

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [14]:
# set_env_securely("MONGODB_URI", "Enter your MongoDB URI: ")

In [15]:
# set_env_securely("VOYAGE_API_KEY", "Enter your Voyage API key: ")

In [16]:
# set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API Key: ")

In [33]:
from langchain_openai import AzureChatOpenAI

# 使用 Azure 时建议关闭 tracing，避免 401（与 zero_to_hero 中一致）
os.environ["OPENAI_AGENTS_DISABLE_TRACING"] = "true"

azure_llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),  # 或直接写 "https://xxx.openai.azure.com/"
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version="2025-03-01-preview",  # 或 "2024-12-01-preview"
    azure_deployment="gpt-4o",          # 你的 deployment 名称
    model="gpt-4o",                     # 模型类型，与 deployment 对应
    temperature=0,
)


In [17]:
MONGODB_URI=os.environ["MONGODB_URI"]
DATABASE_NAME="langmem_agent_memory"
PROCEDURAL_MEMORY_COLLECTION_NAME="procedural_memory"
SEMANTIC_MEMORY_COLLECTION_NAME="semantic_memory"
STATE_CHECKPOINT_COLLECTION_NAME="state_checkpoints"

# Define namespaces for different types of memory
USER_MEMORY_NAMESPACE = ("user_memories",)  # For user-specific memories
KNOWLEDGE_NAMESPACE = ("agent_memory_survey",)  # For PDF content (matches our storage)

In [18]:
from langchain_voyageai import VoyageAIEmbeddings
from langgraph.store.mongodb.base import MongoDBStore, VectorIndexConfig
from pymongo import MongoClient

client = MongoClient(MONGODB_URI)
db = client[DATABASE_NAME]
procedural_collection = db[PROCEDURAL_MEMORY_COLLECTION_NAME]
semantic_collection = db[SEMANTIC_MEMORY_COLLECTION_NAME]

procedural_vector_index_config = VectorIndexConfig(
    dims=1024,
    index_name="procedural_memory_index",
    filters=None,
    fields=None,
    embed=VoyageAIEmbeddings(model="voyage-3-large"),
)

semantic_vector_index_config = VectorIndexConfig(
    dims=1024,
    index_name="semantic_memory_index",
    filters=None,
    fields=None,
    embed=VoyageAIEmbeddings(model="voyage-3-large"),
)

In [19]:
procedural_memory_store = MongoDBStore(
    collection=procedural_collection,
    index_config=procedural_vector_index_config,
    auto_index_timeout=70
)

In [20]:
semantic_memory_store = MongoDBStore(
    collection=semantic_collection,
    index_config=semantic_vector_index_config,
    auto_index_timeout=70,
)

In [21]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.mongodb import MongoDBSaver

checkpointer = MongoDBSaver(
    client=client,
    db_name=DATABASE_NAME, 
    collection_name=STATE_CHECKPOINT_COLLECTION_NAME,
    index_config=procedural_memory_store
)

## Data Ingestion and Setup

Semantic Memory Ingestion

In [22]:
import sys

# Method 1: Add the project root directory to Python path
project_root = "/Users/binzhou/Demo/agent_memory_demo/part1"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utilities.pdf_chunker import ingest_pdf_and_chunk

url = "https://arxiv.org/pdf/2404.13501"

# Ingest and chunk the PDF
chunks = ingest_pdf_and_chunk(url)

Loading PDF with LangChain...
Chunking text...
Successfully created 198 chunks


In [23]:
from utilities.pdf_chunker import store_chunks_in_memory

# Store chunks in procedural memory
store_chunks_in_memory(chunks, semantic_memory_store, KNOWLEDGE_NAMESPACE[0])

# Print first chunk as example
if chunks:
    print("\nFirst chunk example:")
    print(f"Key: {chunks[0]['key']}")
    print(f"Content preview: {chunks[0]['value']['content'][:200]}...")

Storing 198 chunks in procedural memory...
All chunks stored successfully!

First chunk example:
Key: pdf_chunk_0
Content preview: A Survey on the Memory Mechanism of Large
Language Model based Agents
Zeyu Zhang1, Xiaohe Bo1, Chen Ma1, Rui Li1, Xu Chen1, Quanyu Dai2,
Jieming Zhu2, Zhenhua Dong2, Ji-Rong Wen1
1Gaoling School of Ar...


In [24]:
procedural_memory_store.put(("instructions",), key="agent_instructions", value={"prompt": "Write good paper summaries."})

In [25]:
def prompt(state):
    # Get procedural instructions
    item = procedural_memory_store.get(("instructions",), key="agent_instructions")
    instructions = item.value["prompt"]
    
    # Get user query for semantic search
    user_query = state["messages"][-1].content
    
    # Search semantic memory for relevant content
    knowledge_items = []
    try:
        print("Searching semantic memory for relevant content")
        knowledge_items = semantic_memory_store.search(KNOWLEDGE_NAMESPACE, query=user_query, limit=5)
        print(knowledge_items)
    except Exception as e:
        print(f"Warning: Could not search semantic memory: {e}")
    
    # Build system content with instructions and relevant knowledge
    system_content = f"## Instructions\n\n{instructions}\n\n"
    
    if knowledge_items:
        system_content += "## Relevant Knowledge from Agent Memory Research:\n"
        for item in knowledge_items:
            content = item.value.get('content', str(item.value))[:500]  # Limit content length
            system_content += f"- {content}...\n"
        system_content += "\nUse this knowledge to provide informed, accurate responses.\n\n"
    
    sys_prompt = {"role": "system", "content": system_content}
    return [sys_prompt] + state['messages']


In [26]:
def simple_prompt(state):
    """Simple prompt that only includes procedural instructions"""
    item = procedural_memory_store.get(("instructions",), key="agent_instructions")
    instructions = item.value["prompt"]
    sys_prompt = {"role": "system", "content": f"## Instructions\n\n{instructions}\n\nYou have access to search tools for knowledge retrieval when needed."}
    return [sys_prompt] + state['messages']

In [27]:
from langmem import create_manage_memory_tool, create_search_memory_tool

# 🚨 The tools argument can be turned into toolbox memory, a form of procedural memory that can be used to store tools and their descriptions (Use BigTools)
# Create memory management tools for different namespaces
memory_tools = [
    create_manage_memory_tool(USER_MEMORY_NAMESPACE),    # User-specific memories
    create_search_memory_tool(USER_MEMORY_NAMESPACE),    # Search user memories  
    create_search_memory_tool(KNOWLEDGE_NAMESPACE),      # Search PDF knowledge base
]

Agent that uses memory tools 

In [ ]:
agent = create_react_agent(
    azure_llm,
    prompt=prompt, # Prompt for the agent obtained from the database (procedural memory)
    tools=memory_tools,
    store= procedural_memory_store, # Storing the semantic knowledge
    checkpointer=checkpointer, # Storing the state of the agent in the database (procedural memory)
)

/var/folders/0c/6xdkz3fs2m931gvzk6gz44mh0000gp/T/ipykernel_28142/1987237493.py:1: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [29]:
def chat_with_agent(agent, query, thread_id, user_id=None, return_full_result=False):
    """
    Enhanced chat function that supports user-specific memory and can return full results
    
    Args:
        agent: The agent to invoke
        query: User message text
        thread_id: Thread identifier for conversation continuity
        user_id: Optional user identifier for personalized memory
        return_full_result: If True, returns full result state; if False, returns just the content
    
    Returns:
        If return_full_result=True: Full result state with all messages
        If return_full_result=False: Just the last message content
    """
    config = {"configurable": {"thread_id": thread_id}}
    if user_id:
        config["configurable"]["user_id"] = user_id
    
    result_state = agent.invoke(
        {"messages": [{"role": "user", "content": query}]}, 
        config=config
    )
    
    if return_full_result:
        return result_state
    else:
        return result_state["messages"][-1].content



In [30]:

response = chat_with_agent(
    agent=agent, 
    query="Who are the authors of the paper A survey on the memory mechanism of large language models?",
    thread_id="10",
    user_id="user-123",
    return_full_result=True
)

Searching semantic memory for relevant content
[Item(namespace=['agent_memory_survey'], key='pdf_chunk_0', value={'content': 'A Survey on the Memory Mechanism of Large\nLanguage Model based Agents\nZeyu Zhang1, Xiaohe Bo1, Chen Ma1, Rui Li1, Xu Chen1, Quanyu Dai2,\nJieming Zhu2, Zhenhua Dong2, Ji-Rong Wen1\n1Gaoling School of Artificial Intelligence, Renmin University of China, Beijing, China\n2Huawei Noah’s Ark Lab, China\nzeyuzhang@ruc.edu.cn, xu.chen@ruc.edu.cn\nAbstract\nLarge language model (LLM) based agents have recently attracted much attention\nfrom the research and industry communities. Compared with original LLMs, LLM-\nbased agents are featured in their self-evolving capability, which is the basis for\nsolving real-world problems that need long-term and complex agent-environment\ninteractions. The key component to support agent-environment interactions is the\nmemory of the agents. While previous studies have proposed many promising mem-\nory mechanisms, they are scattered 

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: f6764eef********************119b. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [31]:
print(response['messages'][-1].content)

NameError: name 'response' is not defined

In [ ]:
from langmem import create_prompt_optimizer

optimizer = create_prompt_optimizer("openai:gpt-4o")

Understand and explain trajectories

In [ ]:
current_prompt = procedural_memory_store.get(("instructions",), key="agent_instructions").value["prompt"]
feedback = {"request": "Always respond with sentences starting with, according to the paper in question..."}

optimizer_result = optimizer.invoke({"prompt": current_prompt, "trajectories": [(response["messages"], feedback)]})

In [ ]:
print(f"Current prompt: {current_prompt}")
print(f"Optimizer result: {optimizer_result}")


In [ ]:
procedural_memory_store.put(("instructions",), key="agent_instructions", value={"prompt": optimizer_result})


In [ ]:
response = chat_with_agent(
    agent,
    "What is the main point of the paper?",
    "0",
    "user-123",
    return_full_result=True
)

In [ ]:
print(response['messages'][-1].content)

In [ ]:
def eager_prompt(state):
    """Prompt with eager retrieval - no need for store parameter"""
    # Get procedural instructions
    item = procedural_memory_store.get(("instructions",), key="agent_instructions")
    instructions = item.value["prompt"]
    
    # Get user query for semantic search
    user_query = state["messages"][-1].content
    
    # Direct access to semantic memory (no need for store parameter)
    knowledge_items = []
    try:
        knowledge_items = semantic_memory_store.search(KNOWLEDGE_NAMESPACE, query=user_query, limit=3)
    except Exception as e:
        print(f"Warning: Could not search semantic memory: {e}")
    
    # Build system content
    system_content = f"## Instructions\n\n{instructions}\n\n"
    
    if knowledge_items:
        system_content += "## Relevant Knowledge:\n"
        for item in knowledge_items:
            content = item.value.get('content', str(item.value))[:400]
            system_content += f"- {content}...\n"
        system_content += "\n"
    
    sys_prompt = {"role": "system", "content": system_content}
    return [sys_prompt] + state['messages']




In [ ]:
# Create agent with eager retrieval (no store parameter needed for semantic memory)
eager_agent = create_react_agent(
    "openai:gpt-4o",
    prompt=eager_prompt,  # Automatic knowledge injection
    tools=[],  # No search tools needed
    store=procedural_memory_store,  # Only procedural memory in store
    checkpointer=checkpointer,
)

In [ ]:

print("\n=== Approach 2: Eager Retrieval (Always includes relevant knowledge) ===")
query = "Who are the authors of the paper?"
response2 = chat_with_agent(eager_agent, query, "eager-test", return_full_result=False)
print(f"Response: {response2[:200]}...")




In [ ]:
# Install BigTool
%pip install -q langgraph-bigtool


In [ ]:
import uuid
from typing import Literal, Dict, Any
from langmem import create_manage_memory_tool, create_search_memory_tool
from langgraph_bigtool import create_agent
from langgraph.prebuilt import InjectedStore
from langgraph.store.base import BaseStore
from typing_extensions import Annotated

# Define available memory namespaces and their purposes
MEMORY_NAMESPACES = {
    "user_memories": "Store and retrieve user-specific information, preferences, and personal data",
    "agent_memory_survey": "Search the comprehensive research paper about agent memory mechanisms",
    "conversations": "Store and retrieve conversation history and context",
    "user_preferences": "Manage user settings, preferences, and configuration",
    "project_knowledge": "Store and retrieve project-specific knowledge and documentation",
    "research_papers": "General research paper storage and retrieval",
}

def create_dynamic_memory_tool_registry() -> Dict[str, Any]:
    """
    Create a registry of memory tools that can be dynamically retrieved.
    This scales much better than having all tools in an array.
    """
    tool_registry = {}
    
    # Create search and manage tools for each namespace
    for namespace, description in MEMORY_NAMESPACES.items():
        namespace_tuple = (namespace,)
        
        # Search tool for this namespace
        search_tool = create_search_memory_tool(namespace_tuple)
        search_tool.description = f"Search {description.lower()}"
        search_id = f"search_{namespace}"
        tool_registry[search_id] = search_tool
        
        # Manage tool for this namespace  
        manage_tool = create_manage_memory_tool(namespace_tuple)
        manage_tool.description = f"Store/update information in {description.lower()}"
        manage_id = f"manage_{namespace}"
        tool_registry[manage_id] = manage_tool
    
    return tool_registry

# Create the tool registry
memory_tool_registry = create_dynamic_memory_tool_registry()

print(f"Created {len(memory_tool_registry)} memory tools:")
for tool_id, tool in memory_tool_registry.items():
    print(f"  - {tool_id}: {tool.description}")


In [ ]:
# Index memory tools in a store for semantic retrieval
from langchain_voyageai import VoyageAIEmbeddings

# Create a separate store for tool indexing
tool_store = MongoDBStore(
    collection=db["memory_tools"],
    index_config=VectorIndexConfig(
        dims=1024,
        index_name="memory_tools_index", 
        embed=VoyageAIEmbeddings(model="voyage-3-large"),
        filters=None,
        fields=None,
        auto_index_timeout=70
    )
)

# Index each tool with its description for semantic search
for tool_id, tool in memory_tool_registry.items():
    tool_store.put(
        ("memory_tools",),
        tool_id,
        {
            "name": tool.name,
            "description": tool.description,
            "namespace": tool_id.split("_", 1)[1],  # Extract namespace from ID
            "operation": tool_id.split("_", 1)[0],  # Extract operation (search/manage)
        },
    )

print("Memory tools indexed for semantic retrieval")


In [ ]:
def retrieve_memory_tools(
    query: str,
    operation_type: Literal["search", "manage", "both"] = "both",
    *,
    store: Annotated[BaseStore, InjectedStore],
) -> list[str]:
    """
    Intelligently retrieve memory tools based on query and operation type.
    
    Args:
        query: The user's query to determine relevant memory namespaces
        operation_type: Whether to retrieve search tools, manage tools, or both
        store: The injected tool store for searching
    
    Returns:
        List of tool IDs that are relevant to the query
    """
    # Search for relevant tools based on the query
    results = store.search(("memory_tools",), query=query, limit=6)
    
    # Filter by operation type if specified
    tool_ids = []
    for result in results:
        tool_id = result.key
        tool_data = result.value
        
        if operation_type == "both":
            tool_ids.append(tool_id)
        elif operation_type == "search" and tool_data["operation"] == "search":
            tool_ids.append(tool_id)
        elif operation_type == "manage" and tool_data["operation"] == "manage":
            tool_ids.append(tool_id)
    
    # Ensure we always include the research paper search tool for research questions
    research_indicators = ["paper", "research", "author", "study", "survey", "memory mechanism"]
    if any(indicator in query.lower() for indicator in research_indicators):
        if "search_agent_memory_survey" not in tool_ids:
            tool_ids.insert(0, "search_agent_memory_survey")
    
    # Limit to avoid overwhelming the LLM
    return tool_ids[:4]

print("Custom tool retrieval function created")
